In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
import math

C:\Users\nobou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embeddings = np.load('../Data/minilm_mean_vectors.npz')['vectors']
n = embeddings.shape[0]

print(f"Recommended: {math.ceil(math.log2(pow(2*n, 1/3)))}")

Recommended: 7


In [4]:
class L2Hash:
    def __init__(self, dim, r, nbits, seed=1):
        self.seed = seed
        self.nbits = nbits
        
        gen = np.random.RandomState(seed)
        self.a = gen.normal(0, 1, (nbits, dim))
        self.b = gen.uniform(0.0, r)
        self.r = r

    def hash(self, vectors):
        hash_values = (np.dot(vectors, self.a.T) + self.b) / self.r
        hash_binary = (hash_values >= 0).astype(int)
        return np.apply_along_axis(lambda row: ''.join(row.astype(str)), axis=1, arr=hash_binary)

In [5]:
class LSHIndex:
    def __init__(self, dim, r=5, nbits=5, seed=1):
        # Store the vector dimension
        self._dim = dim

        # Created hash codes by applying our projections
        self._hasher = L2Hash(self._dim, r, nbits, seed)
        self._r = r
        self._nbits = nbits
        self._seed = seed
        self._binned_vectors = dict()

    # Sort the hash codes into bins
    def __hashes_to_bins(self, vectors, hash_codes):
        bins_dict = dict()
        unique_bins = np.unique(hash_codes)
        
        for cur_bin in unique_bins:
            bins_dict[cur_bin] = vectors[hash_codes == cur_bin]
        
        return bins_dict

    # Find the closest bins to a hash code by Hamming distance
    def __closest_bins(self, hash_code):
        bins = np.array(list(self._binned_vectors.keys()))
        
        # Calculate Hamming distance
        distances = np.array([sum(c1 != c2 for c1, c2 in zip(hash_code, bin)) for bin in bins])
        sorted_indices = np.argsort(distances)
        return np.array(bins)[sorted_indices]

    # Find the K nearest neighbours for a given bin
    def __find_k_neighbours(self, target, K):
        closest_bins = self.__closest_bins(target)
        neighbours = self._binned_vectors[closest_bins[0]]

        index = 1
        while (neighbours.shape[0] < K and index < len(closest_bins)):
            neighbours = np.concatenate((neighbours, self._binned_vectors[closest_bins[index]]), axis=0)
            index += 1
        
        return neighbours

    # Add function which adds vectors with their given indices to the index
    def add(self, indices, vectors):
        if (self._dim != vectors.shape[1]):
            raise Exception(f"Dimension mismatch: Index ({self._dim}) and vectors ({vectors.shape[1]})")
        
        # Add indexes into vectors (such that we can find the original index after binning)
        indexed_vectors = np.hstack((indices[:, np.newaxis], vectors))
        hash_codes = self._hasher.hash(vectors)
        self._binned_vectors = self.__hashes_to_bins(indexed_vectors, hash_codes)

    # Search function which accepts a vector and a K value (number of results requested)
    def search(self, vector, K=10):
        # Hash the query to its code
        hash_code = self._hasher.hash([vector])[0]
        
        # Find the nearest bins to satisfy K results
        candidate_vectors = self.__find_k_neighbours(hash_code, K)
    
        # Calculate Euclidean distance
        distances = np.sum((candidate_vectors[:,1:] - vector) ** 2, axis=1)
        sorted_indices = np.argsort(distances)[:K]
        
        # Return the indices of the ranked results
        result_indices = candidate_vectors[:,0][sorted_indices]
        return result_indices, distances[sorted_indices]

    # Save function for storing the index as a npz file
    def save(self, path):
        np.savez(
            path,
            properties = {
                "dim": self._dim,
                "r": self._r,
                "seed": self._seed,
                "nbits": self._nbits
            },
            binned_vectors = self._binned_vectors
        )

    # Loading the object from a npz file (to avoid having to rebuild it each time, and to be able to see index size after quantizing)
    @classmethod
    def load(cls, path):
        data = np.load(path, allow_pickle=True)
        instance = cls.__new__(cls)
        properties = data["properties"].item()

        instance._dim = properties["dim"]
        instance._r = properties["r"]
        instance._nbits = properties["nbits"]
        instance._seed = properties["seed"]
        instance._hasher = L2Hash(instance._dim, instance._r, instance._seed, instance._nbits)

        instance._binned_vectors = data["binned_vectors"].item()
        return instance

    # toString method, showcases the distribution of vectors over the bins
    def __str__(self):
        unique_bins = self._binned_vectors.keys()
        return f"LSHIndex ({', '.join([f'bin({unique_bin}) = {len(self._binned_vectors[unique_bin])}' for unique_bin in unique_bins])})"

## Steps (original approach)

- We created an LSHIndex class, it takes vectors, indices, and the number of projections and bins.
- Projections are created, these are used to create hash_codes of each vector.
- These hash_codes are then used to create bin boundaries, and bins are created.
- The hash_codes are then put into these bins

## Steps (p-Stable)

- We didn't have a proper hash function, we just used random projection vectors. 
We implemented the hashing function from p-Stable distributions article.
- We still binned by taking the lowest and highest hash code as the bin boundaries. 
This now resulted in a Gaussian distribution of vectors over bins, obviously.
- We then tried to find a way to distribute the vectors evenly.

- We then looked into the binning strategy (which we misunderstood). 
The resulting hashcodes can be mapped to either 1 or 0, creating a bit. 
We then know a bin a vector belongs to by creating a hash table.

### Notes
- We used euclidean distance

## Redo

- We try out rice rule
- Freedman-diaconis

- Also try for multiple r values
- r = 1, 1.5, 5, 10
- try both rules

## Measure results

- try both rules for getting bins
- 4 different r values (more?)

Implement best rule in index
Apply quantizers on data (query?)
Measure it

In [6]:
index = LSHIndex(embeddings.shape[1], 1.3, 7, 100)
index.add(np.arange(embeddings.shape[0]), embeddings)
print(index)

LSHIndex (bin(0000000) = 1244, bin(0000001) = 1012, bin(0000010) = 2101, bin(0000011) = 8106, bin(0000100) = 1917, bin(0000101) = 1277, bin(0000110) = 3008, bin(0000111) = 10598, bin(0001000) = 1003, bin(0001001) = 909, bin(0001010) = 1574, bin(0001011) = 5137, bin(0001100) = 1316, bin(0001101) = 1044, bin(0001110) = 1714, bin(0001111) = 5334, bin(0010000) = 845, bin(0010001) = 929, bin(0010010) = 2648, bin(0010011) = 11247, bin(0010100) = 1638, bin(0010101) = 2110, bin(0010110) = 3976, bin(0010111) = 20143, bin(0011000) = 1519, bin(0011001) = 1200, bin(0011010) = 2833, bin(0011011) = 8647, bin(0011100) = 2675, bin(0011101) = 2186, bin(0011110) = 5065, bin(0011111) = 14708, bin(0100000) = 1520, bin(0100001) = 1094, bin(0100010) = 1406, bin(0100011) = 3848, bin(0100100) = 4357, bin(0100101) = 2031, bin(0100110) = 2735, bin(0100111) = 8073, bin(0101000) = 1192, bin(0101001) = 784, bin(0101010) = 1226, bin(0101011) = 2867, bin(0101100) = 3272, bin(0101101) = 1492, bin(0101110) = 1841, bin

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

washington_titles = np.load('../Data/washington_idtitle.npz', allow_pickle=True)["title"]
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def find_documents(index, query, K=10):
    query_vector = model.encode(query)
    
    indices = index.search(query_vector, K)[0]
    return washington_titles.iloc[indices]

C:\Users\nobou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
find_documents(index, "American election 2016", K=200)

1292        Two-thirds of Trump voters viewed the election as America’s last chance
1291        Two-thirds of Trump voters viewed the election as America’s last chance
468              Were the polls way off in 2016? A new report offers a mixed answer
3410    The 2016 national polls are looking less wrong after final election tallies
4424                                          Election Essentials: 2016 voter guide
                                           ...                                     
3334      Election recount will take place in Wisconsin, after Stein files petition
3333      Election recount will take place in Wisconsin, after Stein files petition
3332      Election recount will take place in Wisconsin, after Stein files petition
3328      Election recount will take place in Wisconsin, after Stein files petition
3591      Election recount will take place in Wisconsin, after Stein files petition
Name: title, Length: 200, dtype: object